<br>

# Endereços Aleatórios

In [ ]:
import os
import random
import requests

from random import randrange

<br>

## PyCEP

O projeto [PyCEP](https://pypi.org/project/pycep-correios/) possibilita pesquisar CEPs e retornar o lougradouro que representa aquele CEP.
Projeto bastante difundido na internet, porém ainda não possibilita o que eu busco no momento: **pegar faixas de CEP de uma cidade específica**.

In [ ]:
#!pip install pycep-correios

In [ ]:
from pycep_correios import get_address_from_cep, WebService

In [ ]:
cep = '13416-398'

get_address_from_cep(cep, webservice=WebService.APICEP)

In [ ]:
get_address_from_cep(cep, webservice=WebService.VIACEP)

In [ ]:
get_address_from_cep(cep, webservice=WebService.CORREIOS)

<br>

## ViaCEP

O PyCEP, mencionado acima, utiliza a API do [ViaCEP](https://viacep.com.br/) que apresenta diversas funcionalidades a serem pesquisadas (a dos DDD me interessou bastante!).

Há uma ferramenta que achei interessante e "quase" me atende integralmente: trata-se da **Pesquisa de CEP**. Nela é possível inserir:
1. Estado
2. Município
3. Algum termo para "limitar" a pesquisa, sendo esse o ponto que não queria.

In [ ]:
'https://viacep.com.br/ws/sp/santos/senhora/json/'

<br>

## Correios: faixa de CEP

https://buscacepinter.correios.com.br/app/faixa_cep_uf_localidade/index.php

<br>

# Gerando Endereços Aleatórios

In [ ]:
def gera_random_url(estado='sp', municipio='piracicaba'):
    termo = [
        # Padrão
        'rua', 'avenida', 'jardim', 'praça', 'bosque', 

        # Meses
        'janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro',

        # Patentes
        'marechal', 'sargento', 'duque', 'soldado', 'capitão', 'major',

        # Pronomes de Tratamento
        'dom', 'senhor', 'senhora',

        # Nomes
        'almeia', 'caetano', 'pedro', 'barbosa', 'rui', 'pinto', 'joão', 'são',
        ]

    url = 'https://viacep.com.br/ws/{}/{}/{}/json/'.format(estado, municipio, random.choice(termo))    
    return url

In [ ]:
#gera_random_url(estado='sp', municipio='santos')

In [ ]:
def get_list_cep_bairros(estado='sp', municipio='santos', loop=20):
    i = 1
    list_ceps = []
    list_bairros = []
    while i<=loop:
        url = gera_random_url(estado=estado, municipio=municipio)
        resp = requests.get(url)
        json = resp.json()
        for data in json:
            list_ceps.append(data['cep'])
            list_bairros.append(data['bairro'])
        i+=1

    # CEPs
    list_ceps = list(set(list_ceps))
    list_ceps.sort()

    # Bairros
    list_bairros = list(set(list_bairros))
    list_bairros.sort()
    
    print(
        'Município: {}-{}\nCEPs distintos: {}\nBairros distintos: {}'.format(
            municipio.title(), estado.upper(),
            len(list_ceps),
            len(list_bairros)
        )
    )
    return list_ceps, list_bairros

In [ ]:
list_ceps, list_bairros = get_list_cep_bairros(estado='sp', municipio='piracicaba')

In [ ]:
for cep in list_ceps[1:3]:
    # Pega o Endereço Completo via API
    end = get_address_from_cep(cep, webservice=WebService.VIACEP)
    
    # Adiciona um número aleatório
    end['complemento'] = '{}'.format(randrange(1, 999))
    
    # Cria uma chave no dict com Endereço Completo
    end['endereco_completo'] = '{}, {} - {} - {}, {} - CEP: {}'.format(
        end['logradouro'],
        end['complemento'],
        end['bairro'],
        end['cidade'],
        end['uf'],
        end['cep'],
    )
    print(end)